In [2]:
!pip install --no-index --find-links=/dbfs/databricks/libraries/ langchain
!pip install --no-index --find-links=/dbfs/databricks/libraries/ langchain-openai
!pip install --no-index --find-links=/dbfs/databricks/libraries/ python-dotenv

Looking in links: /dbfs/databricks/libraries/
Looking in links: /dbfs/databricks/libraries/
Looking in links: /dbfs/databricks/libraries/


In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI
import os
from dotenv import load_dotenv
from pydantic import BaseModel
from langchain_core.pydantic_v1 import Field
import pandas as pd

/Users/takahashiryusei/.pyenv/versions/3.12.7/envs/playground_py_3_12_7/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
load_dotenv()

# AzureOpenAIのAPI KEY
azure_openai_api_key: str = ''
os.environ['AZURE_OPENAI_ENDPOINT'] = ''

In [ ]:
def get_llm(azure_deployment="gpt-4o-mini", api_version="2024-08-01-preview"):

    llm = AzureChatOpenAI(
        api_key=azure_openai_api_key,
        temperature=1,
        azure_deployment=azure_deployment,
        api_version=api_version,
    )
    return llm

In [ ]:
generate_masking_content_system_prompt = """
【役割】
あなたは日本の個人情報をマスキングするためのデータセットを作成するデータ作成担当者です。
個人情報は、人の氏名、生年月日、会社名、住所、emailアドレス、電話番号です。

【背景】
個人情報をマスキングすることに特化したLLMを作成するために、マスキングされたデータセットが必要
あなたに個人情報をマスキングするためのデータセットを量産してもらいたい

【制約】
- 文章は日本語で、さまざまなバリエーションで文章を生成して下さい。バリエーション豊富であればあるほど良いです。自然な文章で長めの文章を生成して下さい。
- 自然な文章で長文を生成して下さい。
- 文章は下記の【フォーマット】で必ず出力して下さい。「マスク前の文章」と「マスク後の文章」をワンセットで1件だけ生成して下さい。
- 必ず【必須の個人情報】を含めたマスキングデータを作成して下さい。
- 【任意の個人情報】を含めても良いです。自然な文章を作成するにあたり、【任意の個人情報】の内容が必要であれば含めて下さい。
- 【必須の個人情報】と【任意の個人情報】の内容は必ずマスキングして「マスク後の文章:」に出力して下さい。

【必須で含める個人情報】
- 人の氏名

【人の氏名の制約】
- 漢字でフルネームの氏名を生成すること。性と名間にスペースは付けないでください。

【任意で含める個人情報】
- 会社名
- 日本の住所
- emailアドレス
- 電話番号
- 郵便番号
- 生年月日

【フォーマット】
マスキング前の文章:はじめまして、IBM Japan Systems Engineering Co. Ltd.（ISE）の田中太郎です。よろしくお願いします。
-----
マスキング後の文章:はじめまして、<会社名マスク済み>の<氏名マスク済み>です。よろしくお願いします。
"""

set_before_masking_after_masking_system_prompt = """
【役割】
あなたはマスキング前の文章とマスキング後の文章を分けることAIアシスタントです。

【背景】
個人情報をマスキングすることに特化したLLMを作成するために、マスキングされたデータセットが必要です。
マスキング前の文章とマスキング後の文章を明確に分けてほしい。

【制約】
- 「マスク前の文章:」の文章を忠実にbefore_maskingフィールドに格納して下さい。
- 「マスク後の文章:」の文章を忠実にafter_maskingフィールドに格納して下さい。
"""

In [ ]:
class BeforeMaskingAfterMaskingModel(BaseModel):
    before_masking: str = Field(description='マスキング前の文章。「マスキング前の文章:・・・」となっている文章をここのbefore_maskingフィールドに格納する')
    after_masking: str = Field(description='マスキング後の文章。「マスキング後の文章:・・・」となっている文章をここのafter_maskingフィールドに格納する')

    model_config = {
        "json_schema_extra": {
            "required": ["before_masking", "after_masking"]
        }
    }

In [36]:
def generate_masking_content():
    try:
        llm = get_llm()
        user_query = "日本の個人情報をマスキングするためのデータセットを作成したいです。多彩でさまざまなバリエーションの長めの文章を生成して下さい。500文字以上の長めの文章を生成して下さい。"

        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    generate_masking_content_system_prompt
                ),
                ("human", "{input}"),
            ]
        )

        chain = prompt | llm
        result = chain.invoke({
            'input': user_query
        })

        return result

    except Exception as e:
        print(f"タグ生成に失敗しました: {e}")
        return []

In [37]:
def set_before_masking_after_masking(before_masking_after_masking_content):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                'system',
                set_before_masking_after_masking_system_prompt,
            ),
            ("human", "{input}"),
        ]
    )

    llm = get_llm()
    chain = prompt | llm.with_structured_output(BeforeMaskingAfterMaskingModel)
    output_qanda = chain.invoke({
        'input': before_masking_after_masking_content
        })
    
    return output_qanda

In [38]:
before_masking_after_masking_list = []
count = 3

for i in range(count):
    print(f"{i+1}件目のマスキングデータを生成開始")
    before_masking_after_masking_data = generate_masking_content()
    separate_before_masking_after_masking = set_before_masking_after_masking(before_masking_after_masking_data)

    row_data = {
        'before_mask': separate_before_masking_after_masking.before_masking,
        'after_mask': separate_before_masking_after_masking.after_masking
    }
    
    # リストに辞書を追加
    before_masking_after_masking_list.append(row_data)
    print(f"{i+1}件目のマスキングデータを生成完了")

1件目のマスキングデータを生成開始


/Users/takahashiryusei/.pyenv/versions/3.12.7/envs/playground_py_3_12_7/lib/python3.12/site-packages/pydantic/json_schema.py:2270: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='マスキング前の文章。「マスキング前の文章:・・・」となっている文章をここのbefore_maskingフィールドに格納する' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/Users/takahashiryusei/.pyenv/versions/3.12.7/envs/playground_py_3_12_7/lib/python3.12/site-packages/pydantic/json_schema.py:2270: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='マスキング後の文章。「マスキング後の文章:・・・」となっている文章をここのafter_maskingフィールドに格納する' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


1件目のマスキングデータを生成完了
2件目のマスキングデータを生成開始


/Users/takahashiryusei/.pyenv/versions/3.12.7/envs/playground_py_3_12_7/lib/python3.12/site-packages/pydantic/json_schema.py:2270: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='マスキング前の文章。「マスキング前の文章:・・・」となっている文章をここのbefore_maskingフィールドに格納する' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/Users/takahashiryusei/.pyenv/versions/3.12.7/envs/playground_py_3_12_7/lib/python3.12/site-packages/pydantic/json_schema.py:2270: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='マスキング後の文章。「マスキング後の文章:・・・」となっている文章をここのafter_maskingフィールドに格納する' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


2件目のマスキングデータを生成完了
3件目のマスキングデータを生成開始


/Users/takahashiryusei/.pyenv/versions/3.12.7/envs/playground_py_3_12_7/lib/python3.12/site-packages/pydantic/json_schema.py:2270: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='マスキング前の文章。「マスキング前の文章:・・・」となっている文章をここのbefore_maskingフィールドに格納する' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/Users/takahashiryusei/.pyenv/versions/3.12.7/envs/playground_py_3_12_7/lib/python3.12/site-packages/pydantic/json_schema.py:2270: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='マスキング後の文章。「マスキング後の文章:・・・」となっている文章をここのafter_maskingフィールドに格納する' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


3件目のマスキングデータを生成完了


In [39]:
before_masking_after_masking_list

[{'before_mask': '昨日、株式会社テクノソリューションズの新入社員研修で、私、鈴木一郎はチームビルディングアクティビティに参加しました。東京都千代田区丸の内3丁目1-1の本社ビルにて、初めての対面での研修でした。実際に顔を合わせてみると、オンライン会議ではわからなかった細かなニュアンスや交流が生まれ、非常に有意義に感じました。研修終了後には、同僚の山田花子さんや佐藤健太郎さんと一緒に、近くのカフェでメールアドレス（xxxx@example.com）を交換し、今後の連絡手段を確保しました。また、私の生年月日は1990年5月15日で、同僚たちとは同年代なので、親近感を持つことができました。さらに、業務用の電話番号として03-1234-5678を利用することになりましたので、何か困ったことがあればいつでもご連絡ください。これからの業務においても、チームで協力し合いながら成長していきたいと思っています。',
  'after_mask': '昨日、<会社名マスク済み>の新入社員研修で、私、<氏名マスク済み>はチームビルディングアクティビティに参加しました。<住所マスク済み>の本社ビルにて、初めての対面での研修でした。実際に顔を合わせてみると、オンライン会議ではわからなかった細かなニュアンスや交流が生まれ、非常に有意義に感じました。研修終了後には、同僚の<氏名マスク済み>さんや<氏名マスク済み>さんと一緒に、近くのカフェでメールアドレス（<メールアドレスマスク済み>）を交換し、今後の連絡手段を確保しました。また、私の生年月日は<生年月日マスク済み>で、同僚たちとは同年代なので、親近感を持つことができました。さらに、業務用の電話番号として<電話番号マスク済み>を利用することになりましたので、何か困ったことがあればいつでもご連絡ください。これからの業務においても、チームで協力し合いながら成長していきたいと思っています。'},
 {'before_mask': 'はじめまして。佐藤一郎と申します。私は株式会社東京テクノロジーにてソフトウェアエンジニアを務めております。1985年3月14日に東京都新宿区で生まれ、東京都港区六本木に在住しています。仕事の内容としては主にシステム開発やプログラムの設計を担当しており、新しい技術を取り入れることで会社のプロジェクトの効率化に

In [ ]:
# 収集したデータからデータフレームを作成
df = pd.DataFrame(before_masking_after_masking_list)

# CSVに出力
df.to_csv('not_add_space_kanji_full_name.csv', index=False)